## PointPillars

![](./images/20231115210105.png)


### 1.特征编码 Feature Encoder
按照点云数据所在的X，Y轴（不考虑Z轴）将点云数据划分为一个个的网格，凡是落入到一个网格的点云数据被视为其处在一个pillar里，或者理解为它们构成了一个Pillar。

`每个点云用一个D=9维的向量表示，分别为$(x, y, z, r, x_c, y_c, z_c, x_p, y_p)$ `。

其中$x, y, z, r$为该点云的真实坐标信息（三维）和反射强度； 

$x_c, y_c, z_c$为该点云所处Pillar中所有点的几何中心；

$x_p, y_p$为表示该激光点云相对Pillar坐标的偏移量。

假设每个样本中有$P$个非空的pillars，每个pillar中有$N$个点云数据，那么这个样本就可以用一个$(D, P, N)$张量表示。

那么可能就有人问了，怎么保证每个pillar中有$N$个点云数据呢？

如果每个pillar中的点云数据数据超过$N$个，那么我们就随机采样至$N$个；如果每个pillar中的点云数据数据少于$N$个，少于的部分我们就填充为0；

---

下面是openPCDet关于特征编码的实现（openpcdet/pcdet/models/backbones_3d/vfe/pillar_vfe.py）：

In [ ]:
voxel_features, voxel_num_points, coords = batch_dict['voxels'], batch_dict['voxel_num_points'], batch_dict['voxel_coords']

# 计算均值
points_mean = voxel_features[:, :, :3].sum(dim=1, keepdim=True) / voxel_num_points.type_as(voxel_features).view(-1, 1, 1)

# 计算【偏移】xc,yc,zc
f_cluster = voxel_features[:, :, :3] - points_mean

# 创建每个点云到该pillar的坐标中心点偏移量空数据 xp,yp,zp
#  coords是每个网格点的坐标，即[432, 496, 1]，需要乘以每个pillar的长宽得到点云数据中实际的长宽（单位米）
#  同时为了获得每个pillar的中心点坐标，还需要加上每个pillar长宽的一半得到中心点坐标
#  每个点的x、y、z减去对应pillar的坐标中心点，得到每个点到该点中心点的偏移量
f_center = torch.zeros_like(voxel_features[:, :, :3])

# print("voxel_coords", coords[:,0])
f_center[:, :, 0] = voxel_features[:, :, 0] - (coords[:, 3].to(voxel_features.dtype).unsqueeze(1) * self.voxel_x + self.x_offset)
f_center[:, :, 1] = voxel_features[:, :, 1] - (coords[:, 2].to(voxel_features.dtype).unsqueeze(1) * self.voxel_y + self.y_offset)
f_center[:, :, 2] = voxel_features[:, :, 2] - (coords[:, 1].to(voxel_features.dtype).unsqueeze(1) * self.voxel_z + self.z_offset)

# 如果使用绝对坐标，直接组合
if self.use_absolute_xyz: #(True)
    features = [voxel_features, f_cluster, f_center] # 4+3+3=10维
else:
    features = [voxel_features[..., 3:], f_cluster, f_center]
    
# 将特征在最后一维度拼接 得到维度为（M，32,10）的张量
features = torch.cat(features, dim=-1)

## 2.Stacked Pillars --> Learned Features

经过刚刚的处理，顺利得到`Stacked Pillars`。

`Stacked Pillars`到`Learned Features`的转换过程非常简单，可以表述为P*N*D(30000 x 20 x 9)->P*N*C(30000 x 20 x 64)-> P*C(30000*64)。

对应的处理流程可参考代码如下，**`先是经过一个Linear+BN+ReLU`**，然后 **`通过MaxPooling`** 操作将每个Pillar中最大响应的点云提取出来。

上述中的Linear+BN+ReLU可以堆叠，论文使用了最简单的方法。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义PFN类
class PFNLayer(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 use_norm=True,
                 last_layer=False):
        super().__init__()
        
        self.last_vfe = last_layer
        self.use_norm = use_norm
        if not self.last_vfe:
            out_channels = out_channels // 2
        
        # x的维度由（M, 32, 10）升维成了（M, 32, 64）,max pool之后32才去掉
        if self.use_norm: # True
            self.linear = nn.Linear(in_channels, out_channels, bias=False)  # 线性层
            self.norm = nn.BatchNorm1d(out_channels, eps=1e-3, momentum=0.01) # BN层
        else:
            self.linear = nn.Linear(in_channels, out_channels, bias=True)

        self.part = 50000

    def forward(self, inputs):
        if inputs.shape[0] > self.part:
            num_parts = inputs.shape[0] // self.part
            part_linear_out = [self.linear(inputs[num_part*self.part:(num_part+1)*self.part])
                               for num_part in range(num_parts+1)]
            x = torch.cat(part_linear_out, dim=0)
        else:
            x = self.linear(inputs)
        torch.backends.cudnn.enabled = False
        #permute变换维度，(M, 64, 32) --> (M, 32, 64)
          # 这里之所以变换维度，是因为BatchNorm1d在通道维度上进行,对于图像来说默认模式为[N,C,H*W],通道在第二个维度上
        x = self.norm(x.permute(0, 2, 1)).permute(0, 2, 1) if self.use_norm else x
        torch.backends.cudnn.enabled = True
        
        x = F.relu(x) # ReLU
        
        # 完成pointnet的MAXPooling操作，找出每个pillar中最能代表该pillar的点
        x_max = torch.max(x, dim=1, keepdim=True)[0]

        if self.last_vfe:
            return x_max
        else:
            x_repeat = x_max.repeat(1, inputs.shape[1], 1)
            x_concatenated = torch.cat([x, x_repeat], dim=2)
            return x_concatenated

In [ ]:
# 多个PFN结构串联
pfn_layers = []
for i in range(len(num_filters) - 1):
    in_filters = num_filters[i]
    out_filters = num_filters[i + 1]
    pfn_layers.append(
        PFNLayer(in_filters, out_filters, self.use_norm, last_layer=(i >= len(num_filters) - 2))
    )
# 加入线性层，将13维特征变为64维特征
self.pfn_layers = nn.ModuleList(pfn_layers)

...
...

voxel_count = features.shape[1]

# mask中指名了每个pillar中哪些是需要被保留的数据
mask = self.get_paddings_indicator(voxel_num_points, voxel_count, axis=0)

# （M， 32）->(M, 32, 1)
mask = torch.unsqueeze(mask, -1).type_as(voxel_features)

# 将feature中被填充数据的所有特征置0
features *= mask
for pfn in self.pfn_layers:
    features = pfn(features)
    
# (M, 64), 每个pillar抽象出一个64维特征
features = features.squeeze()

## 3.生成伪图像
通过Scatter运算实现的。在openpcdet工程中，实际上在训练数据集制作阶段，就已经生成了坐标位置信息`batch_dict['voxel_coords']`，其维度是P*2,P是Pillar的数量，2是x和y对应的坐标。在Learned Features构造Pseudo Images的时候根据Pillar Index，将Pillar填充到对应的Pseudo Images上。

In [ ]:
class PointPillarScatter(nn.Module):
    def __init__(self, model_cfg, grid_size, **kwargs):
        super().__init__()

        self.model_cfg = model_cfg
        self.num_bev_features = self.model_cfg.NUM_BEV_FEATURES #64
        self.nx, self.ny, self.nz = grid_size # [432,496,1]
        assert self.nz == 1

    def forward(self, batch_dict, **kwargs):
        '''
        batch_dict['pillar_features']-->为VFE得到的数据(M, 64)
        voxel_coords:(M,4) --> (batch_index,z,y,x) batch_index代表了该点云数据在当前batch中的index
        '''
        pillar_features, coords = batch_dict['pillar_features'], batch_dict['voxel_coords']
        batch_spatial_features = []
        # 根据batch_index，获取batch_size大小
        batch_size = coords[:, 0].max().int().item() + 1
        for batch_idx in range(batch_size):
            # 创建一个空间坐标所有用来接受pillar中的数据
            # spatial_feature 维度 (64,214272)
            spatial_feature = torch.zeros(
                self.num_bev_features,
                self.nz * self.nx * self.ny,
                dtype=pillar_features.dtype,
                device=pillar_features.device)

            batch_mask = coords[:, 0] == batch_idx #返回mask，[True, False...]
            this_coords = coords[batch_mask, :] #获取当前的batch_idx的数
            
            #计算pillar的索引，该点之前所有行的点总和加上该点所在的列即可
            indices = this_coords[:, 1] + this_coords[:, 2] * self.nx + this_coords[:, 3]
            indices = indices.type(torch.long)  # 转换数据类型
            pillars = pillar_features[batch_mask, :]
            pillars = pillars.t()
            
            # 在索引位置填充pillar_features
            spatial_feature[:, indices] = pillars
            # 将空间特征加入list,每个元素为(64, 214272)
            batch_spatial_features.append(spatial_feature)

        # 在第0个维度将所有的数据堆叠在一起
        batch_spatial_features = torch.stack(batch_spatial_features, 0)
         # reshape回原空间(伪图像)    （4, 64, 214272）--> (4, 64, 496, 432)
        batch_spatial_features = batch_spatial_features.view(batch_size, self.num_bev_features * self.nz, self.ny, self.nx)
        batch_dict['spatial_features'] = batch_spatial_features
        return batch_dict

<!--
 * @Author: CharlesHAO hao.cheng@wuzheng.com
 * @Date: 2024-03-22 14:38:14
 * @LastEditors: CharlesHAO hao.cheng@wuzheng.com
 * @LastEditTime: 2024-03-22 17:13:50
 * @FilePath: /about_Radar/深度学习篇/目标检测篇/PointPillars/pp_stepBystep.ipynb
 * @Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
-->
## 4.BackBone网络
伪图片作者2D CNN的输入，用来进一步提取图片特征。

从图中可以看出，该2D CNN采用了两个网络。其中一个网络不断缩小特征图的分辨率，同时提升特征图的维度，因此获得了三个不同分辨率的特征图。

另一个网络对 **`三个特征图进行上采样至相同大小，然后进行concatenation。`**

之所以选择这样架构，是因为**不同分辨率的特征图负责不同大小物体的检测。比如分辨率大的特征图往往感受野较小，适合捕捉小物体（在KITTI中就是行人）**。

BackBone模型结构体如下:

第一部分：三个降采样模块
```cmd
 (0): Sequential(
        (0): ZeroPad2d((1, 1, 1, 1))
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (2): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (8): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (9): ReLU()
        (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (11): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (12): ReLU()
      )
      (1): Sequential(
        (0): ZeroPad2d((1, 1, 1, 1))
        (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (2): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (8): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (9): ReLU()
        (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (11): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (12): ReLU()
        (13): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (14): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (15): ReLU()
        (16): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (17): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (18): ReLU()
      )
      (2): Sequential(
        (0): ZeroPad2d((1, 1, 1, 1))
        (1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (2): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (8): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (9): ReLU()
        (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (11): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (12): ReLU()
        (13): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (14): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (15): ReLU()
        (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (17): BatchNorm2d(256, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (18): ReLU()
      )
```

第二部分：三个上采样模块
```
    (deblocks): ModuleList(
      (0): Sequential(
        (0): ConvTranspose2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): Sequential(
        (0): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2), bias=False)
        (1): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (2): Sequential(
        (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(4, 4), bias=False)
        (1): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
```


下面展示代码部分：

In [ ]:


class BaseBEVBackbone(nn.Module):
    def __init__(self, model_cfg, input_channels):
        super().__init__()
        self.model_cfg = model_cfg

        if self.model_cfg.get('LAYER_NUMS', None) is not None:
            assert len(self.model_cfg.LAYER_NUMS) == len(self.model_cfg.LAYER_STRIDES) == len(self.model_cfg.NUM_FILTERS)
            layer_nums = self.model_cfg.LAYER_NUMS
            layer_strides = self.model_cfg.LAYER_STRIDES
            num_filters = self.model_cfg.NUM_FILTERS
        else:
            layer_nums = layer_strides = num_filters = []

        if self.model_cfg.get('UPSAMPLE_STRIDES', None) is not None:
            assert len(self.model_cfg.UPSAMPLE_STRIDES) == len(self.model_cfg.NUM_UPSAMPLE_FILTERS)
            num_upsample_filters = self.model_cfg.NUM_UPSAMPLE_FILTERS
            upsample_strides = self.model_cfg.UPSAMPLE_STRIDES
        else:
            upsample_strides = num_upsample_filters = []

        num_levels = len(layer_nums)
        c_in_list = [input_channels, *num_filters[:-1]]
        
        """"开始搭建模型结构"""
        self.blocks = nn.ModuleList()
        self.deblocks = nn.ModuleList()
        
        for idx in range(num_levels):
            """第一部分：pandding+Conv+BN+ReLU"""
            cur_layers = [
                nn.ZeroPad2d(1),
                nn.Conv2d(
                    c_in_list[idx], num_filters[idx], kernel_size=3,
                    stride=layer_strides[idx], padding=0, bias=False
                ),
                nn.BatchNorm2d(num_filters[idx], eps=1e-3, momentum=0.01),
                nn.ReLU()
            ]
            
            """第二部分，经典结构: Conv+BN+ReLU"""
            for k in range(layer_nums[idx]):
                cur_layers.extend([
                    nn.Conv2d(num_filters[idx], num_filters[idx], kernel_size=3, padding=1, bias=False),
                    nn.BatchNorm2d(num_filters[idx], eps=1e-3, momentum=0.01),
                    nn.ReLU()
                ])
                
            self.blocks.append(nn.Sequential(*cur_layers))
            
            """"第三部分：deblock 上采样"""
            if len(upsample_strides) > 0:
                stride = upsample_strides[idx]
                if stride > 1 or (stride == 1 and not self.model_cfg.get('USE_CONV_FOR_NO_STRIDE', False)):
                    self.deblocks.append(nn.Sequential(
                        nn.ConvTranspose2d(
                            num_filters[idx], num_upsample_filters[idx],
                            upsample_strides[idx],
                            stride=upsample_strides[idx], bias=False
                        ),
                        nn.BatchNorm2d(num_upsample_filters[idx], eps=1e-3, momentum=0.01),
                        nn.ReLU()
                    ))
                else:
                    stride = np.round(1 / stride).astype(np.int32)
                    self.deblocks.append(nn.Sequential(
                        nn.Conv2d(
                            num_filters[idx], num_upsample_filters[idx],
                            stride,
                            stride=stride, bias=False
                        ),
                        nn.BatchNorm2d(num_upsample_filters[idx], eps=1e-3, momentum=0.01),
                        nn.ReLU()
                    ))


## 5. 检测头部分
pointpillars是一种基于锚框Anchor的目标检测方法。

基于锚框（Anchor）的目标检测算法是一种常见的目标检测方法，主要用于在图像中识别和定位多个目标。这种方法的核心思想是在图像中预定义一系列不同形状和大小的锚框，然后通过模型预测这些锚框与真实目标之间的偏移量以及目标的类别。以下是基于锚框的目标检测逻辑的简要概述：

- **预定义锚框**：在图像的不同位置预定义一系列固定大小和形状的锚框。这些锚框被设计为覆盖图像中可能出现的各种目标大小和形状。

- **特征提取**：使用深度学习模型（如卷积神经网络CNN）从输入图像中提取特征。这些特征将用于后续的目标检测和分类。

- **锚框调整和分类**：对于每个锚框，模型会预测两部分内容：

  - 目标检测：预测锚框与其最匹配的真实目标之间的偏移量（通常包括中心点的偏移和宽高的缩放）。这允许模型调整锚框的位置和大小，使其更好地匹配真实目标。

  - 目标分类：预测锚框内包含的目标的类别（如果有的话）。
  
- **非极大值抑制（NMS）**：由于每个目标可能与多个锚框匹配，因此会产生多个重叠的检测框。非极大值抑制是一种后处理步骤，用于在重叠的检测框中选择一个最佳的框，去除其他冗余的框，从而减少重复检测。

- **输出检测结果**：最终，模型输出调整后的锚框位置、大小以及目标的类别，完成目标的检测和分类。


首先看下完整的检测头代码：


In [ ]:
'''
Author: CharlesHAO hcheng1005@gmail.com
Date: 2024-03-22 20:02:44
LastEditors: CharlesHAO hcheng1005@gmail.com
LastEditTime: 2024-03-23 13:42:11
FilePath: /about_Radar/深度学习篇/目标检测篇/PointPillars/pointpillars_step_by_step.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
# 检测头前向传播代码
def forward(self, data_dict):
        spatial_features_2d = data_dict['spatial_features_2d'] # （4，384，248，216）

        # 每个anchor的类别预测-->(4,18,248,216)
        # 4: batch_size
        # 18: 3*2*3, 其中3是类别个数，2是由于每个类别设置两种方向的锚框
        cls_preds = self.conv_cls(spatial_features_2d) 
        
        # 每个anchor的box预测-->(4,42,248,216)
        # 4: batch_size
        # 42: 3*2*7，其中3是类别个数，2是由于每个类别设置两种方向的锚框，7是box属性[x,y,z,dx,dy,dz,theta]
        box_preds = self.conv_box(spatial_features_2d) 

        cls_preds = cls_preds.permute(0, 2, 3, 1).contiguous()  
        box_preds = box_preds.permute(0, 2, 3, 1).contiguous()  # [N, H, W, C] -->(4,248,216,42)
        
        # 将预测结果存入前传结果字典
        self.forward_ret_dict['cls_preds'] = cls_preds
        self.forward_ret_dict['box_preds'] = box_preds
        
        # 如果存在方向卷积层，则继续处理方向
        if self.conv_dir_cls is not None: 
            dir_cls_preds = self.conv_dir_cls(spatial_features_2d) # 每个anchor的方向预测-->(4,12,248,216)
            dir_cls_preds = dir_cls_preds.permute(0, 2, 3, 1).contiguous()  # [N, H, W, C] -->(4,248,216,12)
            self.forward_ret_dict['dir_cls_preds'] = dir_cls_preds
        else:
            dir_cls_preds = None

        if self.training:
            targets_dict = self.assign_targets(gt_boxes=data_dict['gt_boxes']) # （4，39，8）
            self.forward_ret_dict.update(targets_dict)

        # 如果不是训练模式，则直接进行box的预测或对于双阶段网络要生成proposal(此时batch不为1)
        if not self.training or self.predict_boxes_when_training:
            # 输入为最开始的类别和box以及方向的预测，输出为展开后的预测
            batch_cls_preds, batch_box_preds = self.generate_predicted_boxes(
                batch_size=data_dict['batch_size'],
                cls_preds=cls_preds, box_preds=box_preds, dir_cls_preds=dir_cls_preds
            ) 
            data_dict['batch_cls_preds'] = batch_cls_preds # (1, 321408, 3)
            data_dict['batch_box_preds'] = batch_box_preds # (1, 321408, 7)
            data_dict['cls_preds_normalized'] = False

        return data_dict

这段代码是一个目标检测模型。这个过程包括生成类别预测、框（bounding box）预测以及方向预测，并根据是否处于训练模式，进行目标分配或生成最终的预测框。下面是对这个过程的详细解读：

`**输入**`
data_dict: 包含模型输入数据和配置的字典，至少包含spatial_features_2d（空间特征图）和gt_boxes（真实框，仅在训练时使用）。

`**处理流程**`

- `**类别和框预测：**`

    通过卷积层conv_cls和conv_box处理spatial_features_2d来分别生成类别预测和框预测。
    类别预测的形状为(batch_size, 18, height, width)，框预测的形状为(batch_size, 42, height, width)。
    预测结果的形状调整为(batch_size, height, width, channels)以方便后续处理。

- `**方向预测：**`

    如果定义了conv_dir_cls卷积层，则生成每个锚框的方向预测，形状为(batch_size, 12, height, width)。
    同样地，调整预测结果的形状以方便处理。
    
- `**训练模式下的目标分配：**`

    如果处于训练模式，会根据真实框（gt_boxes）为每个锚框分配目标，包括类别标签和回归目标，并更新forward_ret_dict。

- `**预测模式或训练时的预测生成：**`

    如果不是训练模式，或者即使在训练模式下也需要生成预测框（predict_boxes_when_training为True），则调用generate_predicted_boxes函数。
    该函数基于类别预测、框预测和方向预测生成最终的预测框，并将这些预测存储回data_dict中。

`**输出**`

data_dict: 更新后的输入字典，包含生成的预测结果。

### Anchor LOSS
[README](./基于anchor的目标检测流程.md)